### Python spark bulabilmesi için findspark kütüphanesini indirip init ile başlatıyoruz. 

In [117]:
import findspark
findspark.init()

## RDD üzerinde çalışabilmek için SparkContext oluşturmamız gereklidir.

In [118]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

## master içerisinde local[*] yazdığımız için makinemizdeki tüm çekirdekleri kullanacaktır.

In [119]:
spark=SparkSession.builder\
    .master("local[*]")\
    .appName("WordCount")\
    .getOrCreate()

In [120]:
sc=spark.sparkContext

## RomeoJuliet dosyasını okuyoruz.

In [121]:
veri_dosyasi="/Users/gulcanogundur/Downloads/romeojuliet.txt"

In [122]:
shakespeare_rdd=sc.textFile(veri_dosyasi)

In [123]:
shakespeare_rdd.take(100)

['',
 "                    WILLIAM SHAKESPEARE'S",
 '',
 '                       ROMEO & JULIET',
 '',
 '   ADAPTED FOR THE SCREEN BY CRAIG PEARCE AND BAZ LUHRMANN',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '                                       FINAL SHOOTING SCRIPT',
 '',
 '                                             October 6, 1995',
 '',
 '',
 '',
 '',
 'EXT.  HIGHWAY.  AFTERNOON.',
 '',
 'A ribbon of freeway stretching into a blue and pink late',
 'afternoon sky. A huge dark sedan, windows tinted gold,',
 'headlights blazing, powers directly for us.',
 '',
 'CUT TO: A heavy, low-slung, pickup truck traveling toward',
 'the sedan.',
 '',
 'WIDE SHOT: Sky, freeway, the cars closing.',
 '',
 'TIGHT ON: The sedan.',
 '',
 'TIGHT ON: The pickup.',
 '',
 'Like thunderous, jousting opponents, the cars pass in a',
 'deafening cacophony of noise.',
 '',
 'INT.  TRUCK.  AFTERNOON.',
 '',
 'TIGHT ON: 

In [124]:
shakespeare_rdd.count()

6247

## punctuation değerleri hariç tutmak ve tüm değerleri küçük harfe dönüştürmek için aşağıdaki gibi bir fonksiyon yazdık.

In [125]:
def lower_clean_str(x):
  punc='!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~-'
  lowercased_str = x.lower()
  for ch in punc:
    lowercased_str = lowercased_str.replace(ch, '')
  return lowercased_str

In [126]:
shakespeare_rdd = shakespeare_rdd.map(lower_clean_str)

In [127]:
shakespeare_rdd.take(40)

['',
 '                    william shakespeares',
 '',
 '                       romeo  juliet',
 '',
 '   adapted for the screen by craig pearce and baz luhrmann',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '                                       final shooting script',
 '',
 '                                             october 6 1995',
 '',
 '',
 '',
 '',
 'ext  highway  afternoon']

## Tüm satırlardaki değerleri kelime kelime ayırmak için split fonksiyonunu kullanıyoruz.

In [128]:
shakespeare_rdd=shakespeare_rdd.flatMap(lambda satir: satir.split(" "))

## Boşluk olan satırları hariç tutmak için aşağıdaki bir filtreleme yapıyoruz.

In [129]:
shakespeare_rdd = shakespeare_rdd.filter(lambda x:x!='')

In [130]:
shakespeare_rdd.take(4)

['william', 'shakespeares', 'romeo', 'juliet']

## Her bir kelimenin yanında 1 değerini atıyoruz.

In [131]:
shakespeare_count=shakespeare_rdd.map(lambda  kelime:(kelime,1))

In [132]:
shakespeare_count.take(4)

[('william', 1), ('shakespeares', 1), ('romeo', 1), ('juliet', 1)]

## her bir kelimedeki 1 değerlerini toplayarak o kelimenin toplam kaç kez geçtiğini görebiliriz.

In [133]:
shakespeare_count_RBK=shakespeare_count.reduceByKey(lambda x,y:(x+y)).sortByKey()

In [134]:
shakespeare_count_RBK.take(40)

[('1995', 1),
 ('21', 1),
 ('6', 1),
 ('60', 2),
 ('9mm', 2),
 ('a', 563),
 ('abandoned', 1),
 ('able', 1),
 ('about', 3),
 ('above', 12),
 ('abra', 24),
 ('abras', 3),
 ('abroad', 1),
 ('abrupt', 1),
 ('abruptly', 5),
 ('absolved', 1),
 ('abuse', 2),
 ('abuses', 1),
 ('accidentally', 1),
 ('accompanied', 1),
 ('according', 1),
 ('accusation', 1),
 ('accustomed', 2),
 ('ache', 1),
 ('aches', 1),
 ('achingly', 2),
 ('acoustic', 1),
 ('across', 24),
 ('actually', 1),
 ('adagio', 1),
 ('adapted', 1),
 ('address', 1),
 ('addressed', 1),
 ('addresses', 1),
 ('adept', 1),
 ('adieu', 4),
 ('adjacent', 1),
 ('adjoining', 1),
 ('adjust', 1),
 ('admired', 1)]

## Kelime ile kelimenin toplam kaç kez kullanıldığını gösteren değeri yer değiştiriyoruz. Amacımız bir sonraki işlemde 
## en çok geçen kelimeleri sıralayabilmek.

In [135]:
shakespeare_count_RBK=shakespeare_count_RBK.map(lambda x:(x[1],x[0]))

In [136]:
shakespeare_count_RBK.take(5)

[(1, '1995'), (1, '21'), (1, '6'), (2, '60'), (2, '9mm')]

## En çok geçen kelimenin "the" olduğunu görüyoruz. Ancak bu değerler anlam ifade etmeyen stopwords dediğimiz kelimelerdir.


In [137]:
shakespeare_count_RBK.sortByKey(False).take(10)

[(1372, 'the'),
 (563, 'a'),
 (506, 'to'),
 (469, 'of'),
 (464, 'romeo'),
 (461, 'and'),
 (258, 'in'),
 (251, 'juliet'),
 (246, 'is'),
 (224, 'i')]

## Stopwords kelimeleri hariç tutmak için nltk kütüphanesini indiriyoruz ve ingilizce stopwords listesini alıyoruz.

In [138]:
import nltk

In [139]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gulcanogundur/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [140]:
from nltk.corpus import stopwords
stopwords =stopwords.words('english')
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

## stopwords değerlerini çıkardığımızda en çok romeo kelimesinin geçtiğini görüyoruz.

In [141]:
shakespeare_count_RBK = shakespeare_count_RBK.filter(lambda x: x[1] not in stopwords).sortByKey(False)

In [142]:
shakespeare_count_RBK.sortByKey(False).take(20)

[(464, 'romeo'),
 (251, 'juliet'),
 (143, 'mercutio'),
 (133, 'capulet'),
 (114, 'thou'),
 (111, 'benvolio'),
 (111, 'night'),
 (98, 'father'),
 (97, 'ext'),
 (96, 'close'),
 (96, 'nurse'),
 (92, 'cont'),
 (88, 'int'),
 (87, 'cut'),
 (84, 'car'),
 (82, 'love'),
 (81, 'laurence'),
 (79, 'tybalt'),
 (71, 'gloria'),
 (66, 'day')]